# Env

In [ ]:
cd ..

In [6]:
%pip install --upgrade openai

  Attempting uninstall: openai
    Found existing installation: openai 1.57.0
    Uninstalling openai-1.57.0:
      Successfully uninstalled openai-1.57.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tree-of-thoughts-llm 0.1.0 requires openai==0.27.7, but you have openai 1.58.1 which is incompatible.
tree-of-thoughts-llm 0.1.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
tree-of-thoughts-llm 0.1.0 requires sympy==1.12, but you have sympy 1.13.1 which is incompatible.
tree-of-thoughts-llm 0.1.0 requires tqdm==4.65.0, but you have tqdm 4.67.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import json
from tot.prompts.crosswords import propose_prompt, value_prompt
from tot.models import gpt
from tot.tasks.crosswords import MiniCrosswordsEnv

env = MiniCrosswordsEnv()

# Prompt

In [22]:
def prompt_wrap(obs):
    return propose_prompt.format(input=obs)

print(prompt_wrap(env.reset(0)))
# print('---------')
# print(prompt_wrap(env.step('h2. value')[0]))

Let's play a 5 x 5 mini crossword, where each word should have exactly 5 letters.

Current Board:
_____
_____
_____
_____
_____

Unfilled:
h1. An agendum; something to be done: _____
h2. An engine: _____
h3. Pretentious; flowery: _____
h4. A salon; a hall: _____
h5. To mock; to sneer: _____
v1. To heap: _____
v2. An Indian antelope: _____
v3. To intend; to plan; to devise; a nettle; to guess: _____
v4. A nozzle: _____
v5. Desiccator; more dry: _____

Filled:

Changed:


Given the current status, list all possible answers for unfilled or changed words, and your confidence levels (certain/high/medium/low), using the format "h1. apple (medium)". Use "certain" cautiously and only when you are 100% sure this is the correct word. You can list more then one possible answer for each word.



In [23]:
import re
import copy
from tot.models import gpt

def parse_line(input_str):
    # regular expression pattern to match the input string format
    pattern = r'^([hv][1-5])\. ([a-zA-Z]{5,5}) \((certain|high|medium|low)\).*$'

    # use regex to extract the parts of the input string
    match = re.match(pattern, input_str)

    if match:
        # extract the matched groups
        parts = [match.group(1), match.group(2), match.group(3)]
        return parts
    else:
        return None

confidence_to_value = {'certain': 1, 'high': 0.5, 'medium': 0.2, 'low': 0.1}  # TODO: ad hoc

def parse_response(response):
    # split the response into lines
    lines = response.split('\n')

    # parse each line
    parsed_lines = [parse_line(line) for line in lines]

    # filter out the lines that didn't match the format
    parsed_lines = [(line[0].lower() + '. ' + line[1].lower(), confidence_to_value.get(line[2], 0)) for line in parsed_lines if line is not None]

    return parsed_lines if len(parsed_lines) >= 1 else None


def get_candidates_to_scores(env):
    obs = env.render()
    if obs in env.cache: 
        print('cache hit')
        return env.cache[obs]
    print('call gpt')
    responses = gpt(prompt_wrap(obs), model='gpt-3.5-turbo', n=8)
    candidates_to_scores = {}
    for response in responses:
        parsed_response = parse_response(response)
        if parsed_response:
            for candidate, score in parsed_response:
                candidates_to_scores[candidate] = candidates_to_scores.get(candidate, 0) + score
        # choose candiate with highest score
    # print(sorted(candidates_to_scores.items(), key=lambda x: x[1], reverse=True))
    env.cache[obs] = candidates_to_scores
    return candidates_to_scores

def propose_score(env, idx):
    obs = env.reset(idx)
    done = False
    infos = []
    while not done:
        responses = gpt(prompt_wrap(obs), model='gpt-3.5-turbo', n=5)
        candidates_to_scores = {}
        for response in responses:
            parsed_response = parse_response(response)
            if parsed_response:
                for candidate, score in parsed_response:
                    candidates_to_scores[candidate] = candidates_to_scores.get(candidate, 0) + score
        # choose candiate with highest score
        print(sorted(candidates_to_scores.items(), key=lambda x: x[1], reverse=True))
        if len(candidates_to_scores) == 0:
            break
        candidates =  sorted(candidates_to_scores, key=candidates_to_scores.get, reverse=True)
        for candidate in candidates:
            env_ = copy.deepcopy(env)
            env_.step(candidate)
            if not any(_ == 2 for _ in env_.status):
                break
        print(candidate)
        # candidate = input()
        obs, r, done, info = env.step(candidate)
        print(obs)
        print(env.steps, info)
        print('-------------------\n\n\n')
        infos.append(info)
    return infos

# DFS

In [ ]:
def dfs(env, actions, infos, time_limit, prune, max_per_state):
    # get candidate thoughts
    candidates_to_scores = get_candidates_to_scores(env)
    if len(candidates_to_scores) == 0: return 0, [], []
    print(sorted(candidates_to_scores.items(), key=lambda x: x[1], reverse=True))

    # back up current state
    board, status, steps = env.board.copy(), env.status.copy(), env.steps

    # try each candidate
    cnt_per_state = 0
    for action in sorted(candidates_to_scores, key=candidates_to_scores.get, reverse=True):
        obs, r, done, info = env.step(action)
        r = info['r_word']
        if len(infos) < time_limit and env.steps < 10 and not any(_ == 2 for _ in env.status):  # not violating any existing constraints
            cnt_per_state += 1
            if cnt_per_state > max_per_state: break
            count = env.prompt_status()       
            actions.append(action)  

            print(len(infos))
            print(actions)
            print(env.render_board())
            print(info)
            print(count)
            if infos:
                best = max(infos, key=lambda x: x['info']['r_word'])
                print('best', best)
            print('--------------')
            print()

            info = {'total_step': len(infos), 'env_step': env.steps, 'actions': actions.copy(), 'info': info, 'count': count}
            infos.append(info)
            if not prune or count['impossible'] < 1:  # only continue if the current status is possible
                dfs(env, actions, infos, time_limit, prune, max_per_state)
            actions.pop()
        env.reset(env.idx, board=board.copy(), status=status.copy(), steps=steps)

In [24]:
import heapq
import random
def crossword_execute(env, action_info, actions, other_params):
    action = action_info['action']
    if action_info['env_info'] is not None:
        board, status, steps = action_info['env_info']
        env.reset(env.idx, board=board.copy(), status=status.copy(), steps=steps)
    obs, r, done, info = env.step(action)
    r = info['r_word']
    if len(other_params['infos']) < other_params['time_limit'] and env.steps < 10 and not any(_ == 2 for _ in env.status):  # not violating any existing constraints
        
        count = env.prompt_status()
        actions.append(action)  

        print(len(other_params['infos']))
        print(actions)
        print(env.render_board())
        print(info)
        print(count)
        if other_params['infos']:
            best = max(other_params['infos'], key=lambda x: x['info']['r_word'])
            print('best', best)
        print('--------------')
        print()

        info = {'total_step': len(other_params['infos']), 'env_step': env.steps, 'actions': actions.copy(), 'info': info, 'count': count}
        other_params['infos'].append(info)
        if other_params['prune'] and count['impossible'] >= 1:  # only continue if the current status is possible
            return 'non-generate'
    return 'generate'

def crossword_generate(env, action_info, other_params):
    candidates_to_scores = get_candidates_to_scores(env)
    if len(candidates_to_scores) == 0: return []
    sorted_candidates = sorted(candidates_to_scores, key=candidates_to_scores.get, reverse=False)
    sorted_scores = [candidates_to_scores[candidate] for candidate in sorted_candidates]
    print(other_params['max_per_state'])
    if other_params['max_per_state'] is not None and other_params['max_per_state'] > 0:
        sorted_candidates = sorted_candidates[:other_params['max_per_state']]
        sorted_scores = sorted_scores[:other_params['max_per_state']]
    print({ "result_list": [{"action": sorted_candidate, "env_info": (env.board.copy(), env.status.copy(), env.steps)} for sorted_candidate in sorted_candidates], "priority_list": sorted_scores })
    return { "result_list": [{"action": sorted_candidate, "env_info": (env.board.copy(), env.status.copy(), env.steps)} for sorted_candidate in sorted_candidates], "priority_list": sorted_scores }

def sliding_window_sanity_check(sliding_window_size, length_of_queue_stack):
    processed_sliding_window_size = [0, 0]
    if sliding_window_size[0] < 0:
        if sliding_window_size[0] < -length_of_queue_stack:
            processed_sliding_window_size[0] = -length_of_queue_stack
        else:
            processed_sliding_window_size[0] = sliding_window_size[0]
    else:
        if sliding_window_size[0] >= length_of_queue_stack:
            processed_sliding_window_size[0] = length_of_queue_stack - 1
        else:
            processed_sliding_window_size[0] = sliding_window_size[0]

    if sliding_window_size[1] < 0:
        if sliding_window_size[1] < -length_of_queue_stack:
            processed_sliding_window_size[1] = -length_of_queue_stack
        else:
            processed_sliding_window_size[1] = sliding_window_size[1]
    else:
        if sliding_window_size[1] >= length_of_queue_stack:
            processed_sliding_window_size[1] = length_of_queue_stack - 1
        else:
            processed_sliding_window_size[1] = sliding_window_size[1]
    return processed_sliding_window_size

def auto_search(env, other_params, execute_func, generate_func, epsilon = 0.3, decay_rate = 0.9, sliding_window_size = None, heapify_queue_stack = False, queue_stack_valuate_func = None):
    queue_stack = [{'action': None, 'parent_actions': [], 'env_info': None, 'level': 0}]
    while queue_stack:
        if queue_stack_valuate_func is not None:
            queue_stack_valuate_func(env, queue_stack, other_params)
        random_number = random.random()
        if random_number < epsilon:
            sliding_window_indexes = (0, len(queue_stack) - 1) if sliding_window_size is None else sliding_window_size
            if heapify_queue_stack:
                queue_stack_copy = queue_stack.copy()
                heapq.heapify(queue_stack_copy)
                processed_sliding_window_indexes = sliding_window_sanity_check(sliding_window_indexes, len(queue_stack_copy))
                action_info = queue_stack_copy.pop(random.randint(processed_sliding_window_indexes[0], processed_sliding_window_indexes[1]))
                queue_stack.pop(queue_stack.index(action_info))
            else:
                processed_sliding_window_indexes = sliding_window_sanity_check(sliding_window_indexes, len(queue_stack))
                action_info = queue_stack.pop(random.randint(processed_sliding_window_indexes[0], processed_sliding_window_indexes[1]))
        else:
            action_info = queue_stack.pop()
        actions = action_info['parent_actions'].copy()
        level = action_info['level']

        to_generate_thoughts = 'generate'

        if action_info['action'] is not None:
            to_generate_thoughts = execute_func(env, action_info, actions, other_params)

        if to_generate_thoughts == 'generate':
            new_thoughts = generate_func(env, action_info, other_params)
            if (type(new_thoughts) == dict) and 'priority_list' in new_thoughts and new_thoughts['priority_list'] is not None:
                if len(new_thoughts['result_list']) == 0: continue
                for priority, action_info in zip(new_thoughts['priority_list'], new_thoughts['result_list']):
                    queue_stack.append((
                        priority, {
                            'action': action_info['action'], 
                            'parent_actions': actions.copy(), 
                            'env_info': action_info['env_info'],
                            'level': level + 1
                        }
                    ))
            else:
                if len(new_thoughts) == 0: continue
                for action_info in new_thoughts:
                    queue_stack.append({
                        'action': action_info['action'],
                        'parent_actions': actions.copy(), 
                        'env_info': action_info['env_info'],
                        'level': level + 1
                        })
        elif to_generate_thoughts == 'non-generate':
            continue
        elif to_generate_thoughts == 'break':
            break
        epsilon = epsilon * decay_rate

In [25]:
# dfs with pruning
infoss = []
for i in range(0, 100, 5):
    env.reset(i)
    infos = []
    actions = []
    other_params = {'infos': infos, 'time_limit': 100, 'prune': True, 'max_per_state': 3}
    auto_search(env, other_params, crossword_execute, crossword_generate, epsilon=0.3, decay_rate=0.9, sliding_window_size=(-3, -1), heapify_queue_stack=False)
    infoss.append(infos)
    with open('logs/crosswords/infoss_dfs_prune.json', 'w') as fout:
        json.dump(infoss, fout)

call gpt
{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': 'Let\'s play a 5 x 5 mini crossword, where each word should have exactly 5 letters.\n\nCurrent Board:\n_____\n_____\n_____\n_____\n_____\n\nUnfilled:\nh1. An agendum; something to be done: _____\nh2. An engine: _____\nh3. Pretentious; flowery: _____\nh4. A salon; a hall: _____\nh5. To mock; to sneer: _____\nv1. To heap: _____\nv2. An Indian antelope: _____\nv3. To intend; to plan; to devise; a nettle; to guess: _____\nv4. A nozzle: _____\nv5. Desiccator; more dry: _____\n\nFilled:\n\nChanged:\n\n\nGiven the current status, list all possible answers for unfilled or changed words, and your confidence levels (certain/high/medium/low), using the format "h1. apple (medium)". Use "certain" cautiously and only when you are 100% sure this is the correct word. You can list more then one possible answer for each word.\n'}], 'temperature': 0.7, 'max_tokens': 1000, 'n': 8, 'stop': None}
{'model': 'gpt-3.5-turbo', 'message

In [ ]:
# dfs with pruning
infoss = []
for i in range(0, 100, 5):
    env.reset(i)
    infos = []
    actions = []
    dfs(env, actions, infos, 100, prune=True, max_per_state=3)
    infoss.append(infos)
    with open('logs/crosswords/infoss_dfs_prune.json', 'w') as fout:
        json.dump(infoss, fout)

In [ ]:
# dfs with pruning
infoss = []
for i in range(0, 100, 5):
    env.reset(i)
    infos = []
    actions = []
    dfs(env, actions, infos, 100, prune=True, max_per_state=3)
    infoss.append(infos)
    with open('logs/crosswords/infoss_dfs_prune.json', 'w') as fout:
        json.dump(infoss, fout)

In [ ]:
# dfs without pruning
infoss = []
for i in range(0, 100, 5):
    env.reset(i)
    infos = []
    actions = []
    dfs(env, actions, infos, 100, prune=False, max_per_state=3)
    infoss.append(infos)
    with open('logs/crosswords/infoss_dfs_no_prune.json', 'w') as fout:
        json.dump(infoss, fout)

In [3]:
things = [1,2,3]
while things:
    print('HAHAHA', things.pop())


HAHAHA 3
HAHAHA 2
HAHAHA 1


In [16]:
processed_sliding_window_size = (0, 0)

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (3428478268.py, line 1)